# Simple program to compare analytical with numerical solution for Poisson equation 

Note that here we know the exact solution so we use this solution as boundary condition --> method of manufactured solutions

In [1]:
import numpy as np
from mpi4py import MPI
import pyvista
import ufl

from dolfinx import fem
from dolfinx.io import XDMFFile
from dolfinx.fem import FunctionSpace
from dolfinx import plot

from petsc4py.PETSc import ScalarType

## Read the mesh

In [2]:
# Read the mesh
with XDMFFile(MPI.COMM_WORLD, "mesh.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    cell_tags = xdmf.read_meshtags(mesh, name="Grid")
mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)

with XDMFFile(MPI.COMM_WORLD, "facet_mesh.xdmf", "r") as xdmf:
    facet_tags = xdmf.read_meshtags(mesh, name="Grid")

## Plot the mesh

In [3]:
topology, cell_types, geometry = plot.create_vtk_mesh(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

pyvista.set_jupyter_backend("pythreejs")

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

## Finite element function space

In [4]:
# here it is just to define the space, not yet the trial and test functions
V = FunctionSpace(mesh, ("CG", 1))  # Lagrange element and and linear elements (degree 1)

## Boundary conditions

In [5]:
# set uD (Dirichlet BC) to the exact solution, in the discrete function space
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

# apply the boundary values to all degrees of freedom that are on the boundary of the discrete domain
# here facet_tags.values returns the tags for all nodes in the boundary. For example if we have 4 boundaries it will return for each node in a BC the number of the tag between 1 and 4
# here we take all the nodes on the boundary but for example if we only want to apply some condition in boundary 1 we can use facet_tags.indices[facet_tags.values == 1]. So we only have to apply a mask.
boundary_dofs = fem.locate_dofs_topological(V, mesh.topology.dim-1, facet_tags.indices)
bc = fem.dirichletbc(uD, boundary_dofs)

## Trial and test functions

In [6]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

## Source term

In [7]:
# constant value f all over the domain (not only on 1 node)
f = fem.Constant(mesh, ScalarType(-6))

## Variational problem

In [8]:
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = ufl.inner(f, v)* ufl.dx

## Solve the linear system

In [9]:
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

## Compute the error

In [10]:
# first get the analytical solution at each node 
V2 = fem.FunctionSpace(mesh, ("CG", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)
# compute L2 error 
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = np.sqrt(mesh.comm.allreduce(error_local, op=MPI.SUM))
print(f"Error_L2 : {error_L2:.2e}")

Error_L2 : 4.74e-02+0.00e+00j


## Plot the solution uh

In [11]:
u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
u_plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…